# core

> Solveit basic helpers

In [ ]:
#| default_exp core

In [ ]:
gk = tuple((g := globals()).keys())

In [ ]:
#| export
import re
import sys
import inspect
import uuid
from collections import defaultdict
from pathlib import Path
from typing import Any, Mapping
import fastcore.all as FC
import dialoghelper
from dialoghelper.core import _find_frame_dict, add_msg, mk_toollist, find_msg_id, is_usable_tool, read_msg, find_var, update_msg
from fastgit import Git

In [ ]:
from tempfile import TemporaryDirectory
from IPython.display import Markdown, HTML
from fastcore.test import *
from fastcore.xtras import dict2obj
from fastcore.tools import *
import dialoghelper.tmux
import dialoghelper.core
from dialoghelper.stdtools import *
from dialoghelper.inspecttools import *
from dialoghelper.tmux import *
from fastgit import callgit, get_top

In [ ]:
# #| hide
# from nbdev.showdoc import *

# globals - what's available on startup?

Run below first after a restart.

In [ ]:
print(gk)

('__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', 'read_url', 'transient', 'run_cmd', 'sys', '__linked_msgs', 'greet', '__os', '__dialog_name', 'core', 'md_cls_d', 'dh_settings', 'Placements', 'mermaid_url', 'empty', 'add_styles', 'find_var', 'set_var', 'find_dname', 'find_msg_id', 'call_endp', 'curr_dialog', 'msg_idx', 'add_scr', 'iife', 'pop_data', 'fire_event', 'event_get', 'find_msgs', 'view_dlg', 'add_html', 'read_msg', 'read_msgid', 'add_msg', 'del_msg', 'update_msg', 'run_msg', 'copy_msg', 'paste_msg', 'enable_mermaid', 'mermaid', 'toggle_header', 'url2note', 'ast_py', 'ast_grep', 'ctx_folder', 'ctx_repo', 'ctx_symfile', 'ctx_symfolder', 'ctx_sympkg', 'msg_insert_line', 'msg_str_replace', 'msg_strs_replace', 'msg_replace_lines', 'msg_del_lines', 'load_gist', 'gist_file', 'import_string', 'is_usable_tool', 'mk_toollist', 'import_gist', 'tool_inf

In [ ]:
if 'find_msg_id' in gk: assert __msg_id == find_msg_id()
if 'find_dname' in gk: assert __dialog_name == find_dname()
print(f"{g['__name__']=}\n{g['__doc__']=}\n{g['__package__']=}\n{g['__loader__']=}\n{g['__spec__']=}\n{g['transient']=}\n{g['__os']=}\n{g['__dialog_name']=}\n{g['__msg_id']=}")

AssertionError: 

In [ ]:
import ipykernel_helper

In [ ]:
dir(ipykernel_helper)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'core',
 'fix_editable_priority',
 'get_md',
 'gh_blob_to_raw',
 'load_ipython_extension',
 'read_gh_repo',
 'read_url',
 'run_cmd',
 'scrape_url',
 'transient']

In [ ]:
# s = Path('/usr/local/lib/python3.12/site-packages/ipykernel_helper/core.py').read_text()
# add_msg(s, msg_type='raw')

In [ ]:
sys.meta_path

 _frozen_importlib.BuiltinImporter,
 _frozen_importlib.FrozenImporter,
 _frozen_importlib_external.PathFinder,
 __editable___dutil_0_0_1_finder._EditableFinder,
 __editable___pote_0_1_1_finder._EditableFinder,
 __editable___solveit_0_0_68_finder._EditableFinder,

In [ ]:
[k for k in sys.modules if 'solveit' in k]

['__editable___solveit_0_0_68_finder']

In [ ]:
slvt_k = [k for k in sys.modules if '__editable___solveit_' in k][0]
print(dir(slvt := sys.modules[slvt_k]))

['MAPPING', 'ModuleSpec', 'NAMESPACES', 'PATH_PLACEHOLDER', 'Path', 'PathFinder', '_EditableFinder', '_EditableNamespaceFinder', '__annotations__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'annotations', 'chain', 'install', 'module_suffixes', 'spec_from_file_location', 'sys']


In [ ]:
if (msg := read_msg(1, True))['content'].startswith('!cat'):
    update_msg(msg['id'], content=f"!cat {slvt.__file__}")
    run_msg(msg['id'])

In [ ]:
' '.join(_.__module__ for _ in sys.meta_path)

'_distutils_hack _frozen_importlib _frozen_importlib _frozen_importlib_external __editable___dutil_0_0_1_finder __editable___pote_0_1_1_finder __editable___solveit_0_0_68_finder six'

In [ ]:
# s = Path('/usr/local/lib/python3.12/site-packages/__editable___solveit_0_0_68_finder.py').read_text()
# add_msg(s, msg_type='raw')

In [ ]:
#| export
def solveit_version():
    "Return the version of solveit if it is found"
    s = ' '.join(_.__module__ for _ in sys.meta_path)
    mtch = re.match(r'.*__editable___solveit_(\d+)_(\d+)_(\d+)_finder', s)
    return f"{mtch[1]}.{mtch[2]}.{mtch[3]}" if mtch else ''

In [ ]:
solveit_version()

'0.0.68'

In [ ]:
#| export
def in_dialog():
    "Check if the code is running in a solveit dialog"
    return bool(solveit_version() and dialoghelper.find_dname() and dialoghelper.find_msg_id())

In [ ]:
in_dialog()

True

In [ ]:
#| export
def get_caller_globals(): 
    "Return the globals of the caller"
    return inspect.currentframe().f_back.f_globals

In [ ]:
test_var = "I'm in globals"
def caller_func():
    local_var = "I'm local"
    return get_caller_globals()

test_is('test_var' in caller_func(), True)
test_is('local_var' in caller_func(), False)

# helpers

## next dupe in folder

In [ ]:
#| export
def next_dup(fp, marker='_dup'):
    "Get next available duplicate number in fp parent directory"
    fp = Path(fp)
    p, nm, suff = fp.parent, fp.stem.split('.')[0], ''.join(fp.suffixes)
    nm = m[1] if (m := re.match(r'^(.+)'+re.escape(marker)+r'\d+$', nm)) else nm
    # if not p.exists(): i, p/f"{nm}{marker}{i}{suff}"
    dupr = re.compile(r'^'+re.escape(nm+marker)+r'(\d+)$')
    nums = sorted({int(m[1]) for f in p.iterdir() if (m := re.match(dupr, f.stem.split('.')[0]))})
    # if not nums: return 0, p/f"{nm}{suff}"
    for i,n in enumerate(nums, 1):
        if i != n: return i, p/f"{nm}{marker}{i}{suff}"
    return len(nums)+1, p/f"{nm}{marker}{len(nums)+1}{suff}"



A given directory can have files with names that match "*_dup1.*", "*_dup2.*", etc. There can be gaps in the sequence, e.g., 1, 2, 5, 7. `next_dup` get the next available number starting from 1 and considering gaps.

In [ ]:
with TemporaryDirectory() as tmp:
    p = Path(tmp)
    for _ in ('img.jpg', 'img_dup3.jpg'): (p/_).touch()
    n, fp = next_dup(p/'img.jpg')
test_eq((n, fp), (1, p/'img_dup1.jpg'))

In [ ]:
with TemporaryDirectory() as tmp:
    p = Path(tmp)
    for _ in ('img.jpg', 'img_dup1.jpg'): (p/_).touch()
    n, fp = next_dup(p/'img.jpg')
test_eq((n, fp), (2, p/'img_dup2.jpg'))

In [ ]:
with TemporaryDirectory() as tmp:
    p = Path(tmp)
    for _ in ('test_dup1.txt', 'test_dup2.txt', 'test_dup5.txt', 'test_dup7.txt'): (p/_).touch()
    n, fp = next_dup(p/'test_dup2.txt')
test_eq((n, fp), (3, p/'test_dup3.txt'))

In [ ]:
with TemporaryDirectory() as tmp:
    p = Path(tmp)
    (p/'_test_dup23.txt').touch()
    n, fp = next_dup(p/'test_dup3.txt')
test_eq((n, fp), (1, p/'test_dup1.txt'))

In [ ]:
with TemporaryDirectory() as tmp:
    p = Path(tmp)
    for _ in ('img_dup2.jpg', 'img_dup3.jpg'): (p/_).touch()
    n, fp = next_dup(p/'test_dup3.txt')
test_eq((n, fp), (1, p/'test_dup1.txt'))

In [ ]:
#| export
def next_filename(path:str):
    "Get next available duplicate filename for path, e.g., 'img.jpg' -> 'img_dup1.jpg'"
    _,fp = next_dup(Path(path))
    return str(fp)

## simple ids

In [ ]:
#| export
def gen_id(): return f"_{uuid.uuid4().hex[:8]}"

In [ ]:
gen_id()

'_1ea6465b'

## object traversal

In [ ]:
#| export
_empty = inspect.Parameter.empty

In [ ]:
def val_at(o, sym: str, default=_empty, sep='.'):
    "Traverse nested `o` looking for attributes/items specified in dot-separated `sym`."
    if not isinstance(sym, str): raise TypeError(f'{sym=!r} is not a string')
    try:
        for a in sym.split(sep):
            if a[0]=='-' or a[0].isdigit(): a = int(a)
            try: o = o[a]
            except Exception:
                if isinstance(a, int):
                    a = str(a)
                    try: o = o[a]
                    except Exception: pass
                o = getattr(o, a)
    except Exception as e:
        if default is not empty: return default
        raise e
    return o

In [ ]:
records = [
    {'id': 1, 'items': [2, 3], 'meta': {'count': 4}}, 
    {'id': 5, 'items': [6, 7], 'nested': [{'val': 81}, {'val': 82}]}, 
    {'id': 9, 'items': [10, 11], 'meta': {'count': 12}}
]

test_eq(val_at(records, '0.id'), 1)
test_eq(val_at(records, '1.items'), [6, 7])
test_eq(val_at(records, '2.meta'), {'count': 12})

Works with lists of dicts (common in API responses)


In [ ]:
test_fail(lambda: val_at({}, 'a.b'))
test_fail(lambda: val_at([], 'a.b'))
test_fail(lambda: val_at({'a': 1}, 'a.b'))
test_fail(lambda: val_atpath({'a': 1}, 'a', 'b'))

# With default, no error
test_eq(val_at({'a': 1}, 'a.b', None), None)

Error handling: raises when path not found (unless default provided)

In [ ]:
j2 = {
    "app": {
        "Garden": {
            "Flowers": {
                "Red flower": "Rose",
                "White Flower": "Jasmine",
                "Yellow Flower": "Marigold"
            }
        },
        "Fruits": {
            "Yellow fruit": ["Mango", {"Banana": ["Canary Island", "Puerto Rico"]}],
            "Green fruit": "Guava",
            "White Flower": "groovy"
        },
        "Trees": {
            "label": {
                "Yellow fruit": "Pumpkin",
                "White Flower": "Bogan"
            }
        },
        "Numbers": [1, 2, 3, 4, 5],
        "Boolean": True,
        "Null": None
    }
}

j2_str = j2#json.dumps(j2)

test_eq(val_at(j2_str, 'app.Fruits.Yellow fruit.1.Banana.0'), 'Canary Island')
test_eq(val_at(j2_str, 'app.Garden.Flowers.Red flower'), 'Rose')
test_eq(val_at(j2_str, 'app.Numbers.2'), 3)
test_eq(val_at(j2_str, 'app.Boolean'), True)
test_eq(val_at(j2_str, 'app.Null'), None)
test_fail(lambda: val_at(j2_str, 'app.NonExistent'))
test_fail(lambda: val_at(j2_str, 'app.Fruits.Yellow fruit.3'))
test_is(val_at(j2_str, 'app.Fruits.Yellow fruit.3', None), None)

In [ ]:
j2_obj = dict2obj(j2)

test_eq(val_at(j2_obj, 'app.Fruits.Yellow fruit.1.Banana.0'), 'Canary Island')

New version of `val_at` that also acepts indexing, i.e. sym can be dot separated with indexing:
- app.Fruits.Yellow fruit[1].Banana[0] <=> app.Fruits.Yellow fruit.1.Banana.0
- app[Fruits][Yellow fruit][1][Banana][0] <=> app.Fruits.Yellow fruit.1.Banana.0
- app[Fruits].Yellow fruit.1.Banana[0] <=> app.Fruits.Yellow fruit.1.Banana.0
- 2.meta <=> [2].meta
- meta.2 <=> meta[2]

In [ ]:
#| export
def at_(
    o, # Object to traverse (dict, list, object, or nested combination)
    sym: str, # Path using dots and/or brackets (e.g., 'a.b[0].c' or 'a[b][c]')",
    default: Any=_empty, # Value to return if path not found (raises exception if not provided)
    sep='.' # Separator for path segments
) -> Any: # Value at the specified path
    "Traverse nested `o` using path `sym` with dot notation and/or bracket indexing"
    sym = re.sub(r'\[([^\]]+)\]', r'.\1', sym)
    try:
        for a in filter(None, sym.split(sep)):
            if a.lstrip('-').isdigit(): a = int(a)
            try: o = o[a]
            except Exception:
                if isinstance(a, int):
                    try: o = o[str(a)]; continue
                    except Exception: pass
                o = getattr(o, a)
    except Exception:
        if default is not _empty: return default
        raise
    return o

`at_` provides flexible path-based access to nested data structures:

**Supported types:** `o` can be/contains any combination of `Sequence`, `Mapping` (dicts, lists, tuples, L, etc), objects with \_\_getitem__, and/or dataclasses, objects with attributes accesible by `getattr`.

**Path syntax:**
- Dot notation: `'a.b.c'` accesses `o['a']['b']['c']` or `o.a.b.c`
- Bracket notation: `'a[b][c]'` is equivalent to `'a.b.c'`: `[x]` is a shorthand for `.x`
- Mixed: `'a.b[0].c'` combines both styles
- Numeric indices: `'items.2'` or `'items[2]'` for list/array access
- Empty path: `''` returns the object itself

**Access priority:** Item access (`[]`) is tried before attribute access (`.`)

**Error handling:** Raises exception if path not found, unless `default` is provided

In [ ]:
test_eq(at_({'a': 13}, 'a'), 13)

test_eq(at_({'a': {'b': 13}}, 'a.b'), 13)
test_eq(at_({'a': dict2obj({'b': 13})}, 'a.b'), 13)

test_eq(at_({'a': {'3': 7}}, 'a.3'), 7)

test_fail(lambda: at_(o, 'app.NonExistent'))
test_fail(lambda: at_(o, '[0'))
test_fail(lambda: at_(None, 'a'))
test_eq(at_(None, 'a', None), None)
test_fail(lambda: at_(None, 'a'))

test_eq(at_(o := {'meta': [1,2,3]}, ''), o)
test_eq(at_(o, 'meta[2]'), 3)
test_eq(at_(o, 'meta.2'), 3)

test_eq(at_('xyz', 'a', None), None)
test_eq(at_((s := 'xyz'), 'split'), s.split)

test_eq(at_([{'a':1}], '[0][a]'), 1)

test_eq(at_(records, '[2].meta'), {'count': 12})
test_eq(at_(records, '2[meta]'), {'count': 12})

j2_list = [{'a':1}, {'b':2}]
test_eq(at_(j2_list, '0'), {'a':1})
test_eq(at_(j2_list, '1'), {'b':2})
test_eq(at_(j2_list, '[0][a]'), 1)
test_eq(at_(j2_list, '0.a'), 1)
test_eq(at_(j2_list, '[1].b'), 2)

test_eq(at_(j2, 'app[Numbers][2]'), 3)
test_eq(at_(j2, 'app[Fruits][Yellow fruit][1][Banana][0]'), 'Canary Island')
test_eq(at_(j2, 'app.Fruits[Yellow fruit].1.Banana[0]'), 'Canary Island')

test_eq(at_(dict2obj(j2), 'app[Fruits][Yellow fruit][1][Banana][0]'), 'Canary Island')
test_eq(at_(dict2obj(j2), 'app.Fruits[Yellow fruit].1.Banana[0]'), 'Canary Island')

test_is(at_(j2, '[bad][path]', 'default'), 'default')
test_is(at_(j2, '[missing]', None), None)
test_is(at_(j2, 'app[missing]', 'default'), 'default')
test_is(at_(j2, 'app[NonExistent]', None), None)

In [ ]:
print(13)

13


In [ ]:
test_eq(at_(read_msg(), 'output'), '13\n')

I'm a note.

In [ ]:
test_eq(at_(read_msg(), 'output'), '')

In [ ]:
if not at_(read_msg(1), 'content').startswith("I'm a prompt"):
    msgid = add_msg("I'm a prompt", msg_type='prompt', output='are you? really?')
    # _=add_scr(f"selectMsg($('#{msgid}'), {centered: true})")
display(Markdown('### Test markdown'))
print(_msgid := find_msg_id())

### Test markdown

_86e4f755


I'm a prompt

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

are you? really?

In [ ]:
test_eq(at_(read_msg(), 'output'), '<p class="leading-tight">are you? really?</p>\n')

In [ ]:
oo = at_(read_msg(0, True, _msgid), 'output')
oo

'<h3 class="uk-h3">Test markdown</h3>\n\n_86e4f755\n'

Future improvements, more complex paths:

1. **Negative indexing support** - currently `-1` works but only because Python handles it; we could make it explicit
2. **Wildcard/glob patterns** - e.g. `items.*.name` to get all names from a list of dicts
3. **Optional chaining** - e.g. `a?.b?.c` returns `None` instead of raising if any part is missing
4. **Slice support** - e.g. `items[1:3]` or `items[:2]`
5. **Multiple paths** - return values from multiple paths at once
6. **Callable filter** - e.g. `items[lambda x: x['active']].name`

Can we improve `_at` somehow? Give me alternatives. I'm interested in conciseness, strength, flexibility; more complex paths will be handle by another function.

**1. Reduce-based (most concise)**

In [ ]:
# def at_(o, sym, default=_empty, sep='.'):
#     "Traverse nested `o` using path `sym` with dot notation and/or bracket indexing"
#     def step(o, a):
#         if a.lstrip('-').isdigit(): a = int(a)
#         try: return o[a]
#         except: return getattr(o, str(a))
#     try: return reduce(step, filter(None, re.sub(r'\[([^\]]+)\]', r'.\1', sym).split(sep)), o)
#     except: 
#         if default is not _empty: return default
#         # raise

**2. Operator.getitem + getattr combo (cleaner step logic)**

In [ ]:
# def at_(o, sym, default=_empty, sep='.'):
#     "Traverse nested `o` using path `sym` with dot notation and/or bracket indexing"
#     def step(o, a):
#         if a.lstrip('-').isdigit(): a = int(a)
#         for fn in (lambda: o[a], lambda: o[str(a)] if isinstance(a,int) else None, lambda: getattr(o,a)):
#             try: return fn()
#             except: pass
#         raise KeyError(a)
#     try: return reduce(step, filter(None, re.sub(r'\[([^\]]+)\]', r'.\1', sym).split(sep)), o)
#     except:
#         if default is not _empty: return default
#         raise

# add_to_namespace

In [ ]:
#| export
def setup_ns(ns=None, **kwargs):
    "Add `kwargs` to the namespace `ns` or current dialog"
    ns = ns or _find_frame_dict('__msg_id')
    # lc = list(locals().items())[1:]
    thisid = find_msg_id()
    for k,v in kwargs.items(): ns[k] = v
    msgid = link_msg('\n\n'.join(f"{k} = {v}" for k,v in kwargs.items()))
    print(thisid, msgid)

setup_dialog = setup_ns

In [ ]:
setup_ns(
    ORG='', 
    REPO='', 
    USERNAME='civvic', 
    DESCRIPTION='', 
    COMMIT_MESSAGE='', 
    BRANCH_NAME='',
)

_89419261 _8f8b31c7


<!-- linkedto: _89419261 -->
ORG = 

REPO = 

USERNAME = civvic

DESCRIPTION = 

COMMIT_MESSAGE = 

BRANCH_NAME = 

In [ ]:
test_is('USERNAME' in globals(), True)

# info

In [ ]:
g = Git('.')

In [ ]:
g('status')

['On branch dev',
 'Changes not staged for commit:',
 '  (use "git add <file>..." to update what will be committed)',
 '  (use "git restore <file>..." to discard changes in working directory)',
 '\tmodified:   ../explorer/conn.ipynb',
 '\tmodified:   00_core.ipynb',
 '',
 'Untracked files:',
 '  (use "git add <file>..." to include in what will be committed)',
 '\t../explorer/isolated.ipynb',
 '',
 'no changes added to commit (use "git add" and/or "git commit -a")']

In [ ]:
g('status', '-bs')

['## dev',
 ' M ../explorer/conn.ipynb',
 ' M 00_core.ipynb',
 '?? ../explorer/isolated.ipynb']

In [ ]:
#| export
def info():
    "Returns information about the dialog"
    ver = f'''Solveit version: **{solveit_version()}**  
dialoghelper version: **{dialoghelper.__version__}**  
'''
    gs = ''
    g = Git('.')
    if g.exists:
        br, *chngs = g('status', '-bs')
        gs = f'''git branch: **{br.split()[-1]}**  
git changes: {chngs}  
'''
    return ver + gs

In [ ]:
Markdown(info())

Solveit version: **0.0.68**  
dialoghelper version: **0.1.8**  
git branch: **dev**  
git changes: [' M ../explorer/conn.ipynb', ' M 00_core.ipynb', '?? ../explorer/isolated.ipynb']  


In [ ]:
#| export
def add_info(msgid:str=''):
    "Add a message with information about the dialog"
    return link_msg(info(), id=msgid)

In [ ]:
add_info();

<!-- linkedto: _f40fb050 -->
Solveit version: **0.0.68**  
dialoghelper version: **0.1.8**  
git branch: **dev**  
git changes: [' M ../explorer/conn.ipynb', ' M 00_core.ipynb', '?? ../explorer/isolated.ipynb']  


# summarize

In [ ]:
#| export
def summarize(target, context): pass

# Messages

## get output

In [ ]:
find_var??


```python
def find_var(var:str):
    "Search for var in all frames of the call stack"
    return _find_frame_dict(var)[var]
```

**File:** `/usr/local/lib/python3.12/site-packages/dialoghelper/core.py`

In [ ]:
code_id = (msg := read_msg()).id
msg.output[:100]

'<div class="relative"><div><div data-code=\'def find_var(var:str):\n    "Search for var in all frames '

In [ ]:
2;

In [ ]:
code_id = (msg := read_msg()).id
msg.output

''

In [ ]:
display(HTML('asdf'))

In [ ]:
html_id = (msg := read_msg()).id
print(msg.output)

asdf


a note

In [ ]:
note_id = (msg := read_msg()).id
print(repr(msg.output))

''


a prompt

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

a response

In [ ]:
prompt_id = (msg := read_msg()).id
print(repr(msg.output))

'<p class="leading-tight">a response</p>\n'


In [ ]:
raw_id = (msg := read_msg()).id
print(repr(msg.output))

''


In [ ]:
#| export
FC.delegates(read_msg)
def get_output(id:str=None, **kwargs) -> list[str]:
    msg = read_msg(0, id=id or find_msg_id())
    return msg.output

In [ ]:
get_output(prompt_id)

'<p class="leading-tight">a response</p>\n'

## message linking

see [linked_msg](/dialog_?name=vic%2Fdutil%2Fexplorer%2Flinked_msg)

In [ ]:
#| export
_ts, _te = ('<!-- ', '#| '), (' -->', '')
def get_tag(name:str, msgid:str='', msg_type:str='note'):
    return f"{_ts[msg_type=='code']}{name}: {msgid or find_msg_id()}{_te[msg_type=='code']}"

In [ ]:
get_tag('tag')

'<!-- tag: _e6275542 -->'

In [ ]:
get_tag('linked', '_asdfqwer')

'<!-- linked: _asdfqwer -->'

In [ ]:
get_tag('linked', '_asdfqwer', 'code')

'#| linked: _asdfqwer'

In [ ]:
#| export
get_ipython().xpush(__linked_msgs={})  # WARNING: neither get_ipython (in user_ns) nor ipykernel_helper.xpush is documented

In [ ]:
#| export
@FC.delegates(add_msg)
def link_msg(
    content:str=None,  # Content of the linked message
    id:str=None,  # ID of the message to link to, or current message if not provided
    **kwargs  # Additional keyword arguments for `add_msg` or `update_msg`
    ):  # id of linked message
    "Add or update a message linked to `anchor` message. Note only one linked msg per anchor."
    anchor_id, linked = id or find_msg_id(), find_var('__linked_msgs')
    if linked_id := linked.get(anchor_id):
        if 'msg' not in (msg := read_msg(0, True, linked_id)):
            tag = get_tag('linkedto', anchor_id, kwargs.get('msg_type', msg.msg_type))
            if content: kwargs['content'] = f"{tag}\n{content}"
            linked[anchor_id] = update_msg(linked_id, **kwargs)
            return linked[anchor_id]
    tag = get_tag('linkedto', anchor_id, kwargs.get('msg_type', 'note'))
    linked[anchor_id] = add_msg(f"{tag}\n{content or ''}", id=anchor_id, **kwargs)
    return linked[anchor_id]

In [ ]:
def hydrate():
    pass

# tools

In [ ]:
dialoghelper.__version__

'0.1.8'

In [ ]:
# fc_tool_info()

Tools available from fastcore.tools: &`[rg, sed, view, create, insert, str_replace, strs_replace, replace_lines, move_lines]`

In [ ]:
# tool_info()

Tools available from dialoghelper: &`[curr_dialog, view_dlg, msg_idx, find_msgs, read_msg, del_msg, add_msg, update_msg, msg_insert_line, msg_str_replace, msg_strs_replace, msg_replace_lines]`

In [ ]:
# inspect_tool_info()

Tools available from inspecttools: &`[symsrc,showsrc,gettype,getdir,doimport,getval,getnth,symlen,symslice,symsearch,symset,run_code_interactive,tracetool]`

In [ ]:
# dialoghelper.tmux.tmux_tool_info()

dialoghelper.tmux tools: &`[pane, list_panes, panes, list_windows, windows, list_sessions, sessions]`

In [ ]:
def get_tool_names(ns): return [k for k,v in ns.items() if callable(v)]

In [ ]:
# print(list(get_tool_names(globals())))

see [get_tool_names isolated](/dialog_?name=vic%2Fdutil%2Fexplorer%2F00_core_isolated)

In [ ]:
#| export
def get_tool_names(
    ns:Mapping=None,  # module or mapping; None uses IPython user namespace
    exclude:Mapping|list[str]=None,  # module/mapping (recursively scanned) or list of symbol names to exclude
    only_exported:bool=False,  # if ns is a module, only include symbols in __all__
    exclude_private:bool=True  # exclude symbols starting with '_'
) -> dict[str,list[str]]:  # module name -> list of tool names
    "Return dict mapping module names to lists of usable tool names from namespace ns (or IPython user namespace if None)."
    exports = set(getattr(ns, '__all__', []))
    if inspect.ismodule(ns): ns = vars(ns)
    if not ns: ns = get_ipython().user_ns
    if exclude: exclude = set(sum(get_tool_names(exclude).values(), []) if not FC.is_listy(exclude) else exclude)
    res = defaultdict(list)
    for k,v in ns.items():
        if exclude_private and k[0] == '_': continue
        if only_exported and k not in exports: continue
        if exclude and k in exclude: continue
        if not hasattr(__builtins__, k) and callable(v):
            try:
                if is_usable_tool(v): 
                    if inspect.isclass(v) and '__call__' not in v.__dict__: continue
                    res[getattr(v, '__module__', 'unknown')].append(k)
            except Exception: pass
    return dict(res)

In [ ]:
#| export
FC.delegates(get_tool_names)
def show_tool_names(*args, **kwargs):
    for mn,syms in get_tool_names(*args, **kwargs).items():
        print(mn)
        print('  ', ', '.join(syms))

In [ ]:
show_tool_names()

ipykernel_helper.core
   read_url
fastcore.tools
   run_cmd, rg, sed, view, create, insert, str_replace, strs_replace, replace_lines, move_lines, get_callable
vic.vic
   greet
dialoghelper.core
   add_styles, find_var, find_msg_id, curr_dialog, iife, find_msgs, view_dlg, add_html, read_msg, read_msgid, add_msg, del_msg, update_msg, run_msg, copy_msg, paste_msg, toggle_header, url2note, ast_py, ast_grep, msg_insert_line, msg_str_replace, msg_strs_replace, msg_replace_lines, msg_del_lines, load_gist, gist_file, is_usable_tool, import_gist
dialoghelper.inspecttools
   doimport, resolve, symsrc, showsrc, gettype, getdir, getval, getnth, symlen, symslice, symsearch, symset, run_code_interactive, tracetool
dialoghelper.tmux
   shell_ret, set_default_history, pane, list_panes, panes, list_windows, windows, list_sessions, sessions, tmux_tool_info
__main__
   solveit_version, in_dialog, get_caller_globals, link_msg, info, add_info, get_tool_names


In [ ]:
show_tool_names(dialoghelper.stdtools)

dialoghelper.core
   add_styles, find_var, find_msg_id, curr_dialog, iife, find_msgs, view_dlg, add_html, read_msg, read_msgid, add_msg, del_msg, update_msg, run_msg, copy_msg, paste_msg, toggle_header, url2note, ast_py, ast_grep, msg_insert_line, msg_str_replace, msg_strs_replace, msg_replace_lines, msg_del_lines, load_gist, gist_file, is_usable_tool, import_gist
dialoghelper.inspecttools
   doimport, resolve, symsrc, showsrc, gettype, getdir, getval, getnth, symlen, symslice, symsearch, symset, run_code_interactive, tracetool
fastcore.tools
   run_cmd, rg, sed, view, create, insert, str_replace, strs_replace, replace_lines, move_lines, get_callable


In [ ]:
show_tool_names(dialoghelper.core, only_exported=True)

dialoghelper.core
   add_styles, find_var, find_msg_id, curr_dialog, iife, find_msgs, view_dlg, add_html, read_msg, read_msgid, add_msg, del_msg, update_msg, run_msg, copy_msg, paste_msg, toggle_header, url2note, ast_py, ast_grep, msg_insert_line, msg_str_replace, msg_strs_replace, msg_replace_lines, msg_del_lines, load_gist, gist_file, is_usable_tool, import_gist


In [ ]:
show_tool_names(dialoghelper.tmux)

dialoghelper.core
   add_msg
builtins
   WrapperDescriptorType, MethodWrapperType, MethodDescriptorType, BuiltinFunctionType, BuiltinMethodType
fastcore.imports
   ipython_shell, in_ipython, in_colab, in_jupyter, in_notebook
fastcore.basics
   NullType, num_cpus, str2float, str2list, str2date
fastcore.xtras
   walk, globtastic, pglob, loads_multi, parse_env, rtoken_hex, modify_exception, stringfmt_names, utc2local, local2utc, console_help, type2str, is_typeddict, reawaitable
toolslm.funccall
   python
dialoghelper.tmux
   shell_ret, set_default_history, pane, list_panes, panes, list_windows, windows, list_sessions, sessions, tmux_tool_info


In [ ]:
#| export
def add_tools_card(ns=None):
    "Add a message with all tools in namespace `ns` or caller globals"
    ns = ns or get_ipython().user_ns
    mod2tool = get_tool_names(ns)
    content = '\n\n'.join(f"## {mod}\n\n{mk_toollist(ns[t] for t in tools)}" for mod,tools in mod2tool.items())
    link_msg(content)

In [ ]:
add_tools_card()

<!-- linkedto: _9bd38272 -->
## ipykernel_helper.core

- &`read_url`: Read url from web

## fastcore.tools

- &`run_cmd`: Run `cmd` passing split `argstr`, optionally checking for allowed argstr
- &`rg`: Run the `rg` command with the args in `argstr` (no need to backslash escape)
- &`sed`: Run the `sed` command with the args in `argstr` (e.g for reading a section of a file)
- &`view`: View directory or file contents with optional line range and numbers
- &`create`: Creates a new file with the given content at the specified path
- &`insert`: Insert new_str at specified line number
- &`str_replace`: Replace first occurrence of old_str with new_str in file
- &`strs_replace`: Replace for each str pair in old_strs,new_strs
- &`replace_lines`: Replace lines in file using start and end line-numbers (index starting at 1)
- &`move_lines`: Move lines from start_line:end_line to before dest_line
- &`get_callable`: Return callable objects defined in caller's module

## vic.vic

- &`greet`: Add a note with `wave` message, and code message with setup python path to the current dialog

## dialoghelper.core

- &`add_styles`: Add solveit styles to `s`
- &`find_var`: Search for var in all frames of the call stack
- &`find_msg_id`: Get the message id by searching the call stack for __msg_id.
- &`curr_dialog`: Get the current dialog info.
- &`iife`: Wrap javascript code string in an IIFE and execute it via `add_html`
- &`find_msgs`: Find `list[dict]` of messages in requested dialog that contain the given information. Call with no args to see the full dialog. Often it is more efficient to call `view_dlg` to see the whole dialog at once, so you can use it all from then on, instead of using `find_msgs`.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
    Message ids are identical to those in LLM chat history, so do NOT call this to view a specific message if it's in the chat history--instead use `read_msgid`.
    Note that LLM chat history only includes messages above the current prompt, whereas `find_msgs` can access *all* messages.
    To refer to a found message from code or tools, use its `id` field.
- &`view_dlg`: Concise XML view of all messages (optionally filtered by type), not including metadata. Often it is more efficient to call this to see the whole dialog at once (including line numbers if needed), instead of running `find_msgs` or `read_msg` multiple times.
- &`add_html`: Send HTML to the browser to be swapped into the DOM
- &`read_msg`: Get the message indexed in the current dialog.
    NB: Messages in the current dialog above the current message are *already* visible; use this only when you need line numbers for editing operations, or for messages not in the current dialog or below the current message.
    - To get the exact message use `n=0` and `relative=True` together with `id`.
    - To get a relative message use `n` (relative position index).
    - To get the nth message use `n` with `relative=False`, e.g `n=0` first message, `n=-1` last message.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
- &`read_msgid`: Get message `id`. Message IDs can be view directly in LLM chat history/context, or found in `find_msgs` results.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
- &`add_msg`: Add/update a message to the queue to show after code execution completes.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
- &`del_msg`: Delete a message from the dialog. DO NOT USE THIS unless you have been explicitly instructed to delete messages.
- &`update_msg`: Update an existing message. Provide either `msg` OR field key/values to update.
    - Use `content` param to update contents.
    - Only include parameters to update--missing ones will be left unchanged.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
- &`run_msg`: Adds a message to the run queue. Use read_msg to see the output once it runs.
- &`copy_msg`: Add `ids` to clipboard.
- &`paste_msg`: Paste clipboard msg(s) after/before the current selected msg (id).
- &`toggle_header`: Toggle collapsed header state for `id`
- &`url2note`: Read URL as markdown, and add note(s) below current message with the result
- &`ast_py`: Get an SgRoot root node for python `code`
- &`ast_grep`: Use `ast-grep` to find code patterns by AST structure (not text).
    
    Pattern syntax:
    - $VAR captures single nodes, $$$ captures multiple
    - Match structure directly: `def $FUNC($$$)` finds any function; `class $CLASS` finds classes regardless of inheritance
    - DON'T include `:` - it's concrete syntax, not AST structure
    - Whitespace/formatting ignored - matches structural equivalence
    
    Examples: `import $MODULE` (find imports); `$OBJ.$METHOD($$$)` (find method calls); `await $EXPR` (find await expressions)
    
    Useful for: Refactoring—find all uses of deprecated APIs or changed signatures; Security review—locate SQL queries, file operations, eval calls; Code exploration—understand how libraries are used across codebase; Pattern analysis—find async functions, error handlers, decorators; Better than regex—handles multi-line code, nested structures, respects syntax
- &`msg_insert_line`: Insert text at a specific line number in a message. Be sure you've called `read_msg(..., nums=True)` to ensure you know the line numbers.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
- &`msg_str_replace`: Replace first occurrence of old_str with new_str in a message.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
- &`msg_strs_replace`: Replace multiple strings simultaneously in a message.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
- &`msg_replace_lines`: Replace a range of lines with new content in a message.
    Be sure you've called `read_msg(..., nums=True)` to ensure you know the line numbers.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
- &`msg_del_lines`: Delete a range of lines from a message. Be sure you've called `read_msg(..., nums=True)` to ensure you know the line numbers.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
- &`load_gist`: Retrieve a gist
- &`gist_file`: Get the first file from a gist
- &`is_usable_tool`: True if the function has a docstring and all parameters have types, meaning that it can be used as an LLM tool.
- &`import_gist`: Import gist directly from string without saving to disk

## dialoghelper.inspecttools

- &`doimport`: Import a module into the caller's global namespace
- &`resolve`: Resolve a dotted symbol string to its Python object, with optional [n] indexing. Sets global `_last` to the resolved object for chaining.
    Pass `"_last"` to reference the result of the previous tool call.

    Examples:

    - `resolve("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `resolve("mylist[2]")` -> third element of mylist
- &`symsrc`: Get the source code for a symbol.

    Examples:

    - `symsrc("sympy.sets.sets.Interval")` -> source code of Interval class
    - `symsrc("_last")` -> source of object from previous tool call
    - For dispatchers or registries of callables: `getnth("module.dispatcher.funcs", n) then symsrc("_last")`
- &`showsrc`: Create a note to show the user (and following LLM prompts) the source of `sym`, following `symsrc` rules
- &`gettype`: Get the type of a symbol and set `_last`.

    Examples:

    - `gettype("sympy.sets.sets.Interval")` -> `<class 'type'>`
    - `gettype("_last")` -> type of previous result
- &`getdir`: Get dir() listing of a symbol's attributes and set `_last`. E.g: `getdir("sympy.Interval")` -> `['__add__', '__and__', ...]`
- &`getval`: Get repr of a symbol's value and set `_last`.

    Examples:
    
    - `getval("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `getval("some_dict.keys")` -> `dict_keys([...])`
- &`getnth`: Get the nth value from a dict (or any object with .values()). Sets `_last` so you can chain with `symsrc("_last")` etc.

    Examples:
    
    - `getnth("dispatcher.funcs", 12)` -> 13th registered function
    - `getnth("dispatcher.funcs", 0); symsrc("_last")` -> source of first handler
- &`symlen`: Returns the length of the given symbol
- &`symslice`: Returns the contents of the symbol from the given start to the end.
- &`symsearch`: Search contents of symbol, which is assumed to be str for regex, or iterable for non-regex.
    Regex mode returns (match, start, end) tuples; otherwise returns (item, index) tuples
- &`symset`: Set _ai_sym to the given value
- &`run_code_interactive`: Insert code into user's dialog and request for the user to run it. Use other tools where possible, 
    but if they can not find needed information, *ALWAYS* use this instead of guessing or giving up.
    IMPORTANT: This tool is TERMINAL - after calling it, you MUST stop all tool usage 
    and wait for user response. Never call additional tools after this one.
- &`tracetool`: 
    Trace execution using sys.monitoring (Python 3.12+), returning a list of per-call traces.

    Return:
    - list of length <= 10
    - one element per call to `target_func` (including recursion)
    - each element is: (stack_str, trace_dict)
        stack_str: call stack string (filtered so `fn` is the shallowest frame shown)
        trace_dict: {
        "<source snippet for AST-line>": ( hit_count, { "var": [ (type_name, truncated_repr), ... up to 10 ], ... } ),
        ...}

    Semantics:
    - "Line" means an AST-level line: separate statements (even if on one physical line via `;`).
    - Compound statements are keyed by their header only.
    - Unchanged variables → `('type', 'repr')` tuple, changed variables → `[('type', 'repr'), ...]` list.
    - Comprehensions are treated as a line node and are monitored, including inside the comprehension frame, with per-iteration snapshots.
    - Snapshots are recorded after each line finishes, so assignments show updated values.
    

## dialoghelper.tmux

- &`shell_ret`: Run shell command locally or over ssh (use host for alias, or ip/user/keyfile)
- &`set_default_history`: Set the default number of lines to capture from tmux history
- &`pane`: Grab the tmux history in plain text
- &`list_panes`: List panes for a session/window (or current if none specified)
- &`panes`: Grab history from all panes in a session/window
- &`list_windows`: List all windows in a session
- &`windows`: Grab history from all panes in all windows of a session
- &`list_sessions`: List all tmux sessions
- &`sessions`: Grab history from all panes in all windows of all sessions
- &`tmux_tool_info`: Add a the message below, showing the available tools, to Solveit

## __main__

- &`solveit_version`: Return the version of solveit if it is found
- &`in_dialog`: Check if the code is running in a solveit dialog
- &`get_caller_globals`: Return the globals of the caller
- &`link_msg`: Add or update a message linked to `anchor` message. Note only one linked msg per anchor.
- &`info`: Returns information about the dialog
- &`add_info`: Add a message with information about the dialog
- &`get_tool_names`: Return dict mapping module names to lists of usable tool names from namespace ns (or IPython user namespace if None).

# export -

In [ ]:
from dutil.flakes import show_flakes
show_flakes()

No warnings to report

In [ ]:
# #| hide
# import nbdev; nbdev.nbdev_export()

In [ ]:
# #|hide
# #|eval: false
# import fastcore.all as FC
# from nbdev import nbdev_export
# if FC.IN_NOTEBOOK:
#     nb_path = '00_core.ipynb'
#     nbdev_export(nb_path)